# Wet vegetation threshold tool

This notebook implements a widget for helping fihd good thresholds for wet vegetation.

In [1]:
import asyncio
from time import time

import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
import xarray
from skimage.filters import threshold_otsu

In [2]:
def get_tcw(tile_label):
    tif_path_base = "/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_wetness/LS_TCW_PC_{}_1986_2020_summary.tif"
    tif_path = tif_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_rasterio(tif_path)
    return da

def get_mrvbf(tile_label):
    mrvbf_path_base = (
        mrvbf_base_path
    ) = "/g/data/u46/wofs/confidence_albers/MrVBF/tiles/mrvbf_{}.nc"
    mrvbf_path = mrvbf_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_dataset(mrvbf_path)
    return da

def get_tcg(tile_label):
    tif_path_base = "/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_{}_1986_2020_summary.tif"
    tif_path = tif_path_base.format(tile_label.replace(",", "_"))
    da = xarray.open_rasterio(tif_path)
    return da

In [3]:
%%bash
ls /g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/*.tif

/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-15_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-17_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-19_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-22_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_10_-32_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-16_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-29_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_12_-32_1986_2020_summary.tif
/g/data/r78/datacube_stats/TCW_stats/demo_tiles/output_qld_greenness/LS_TCG_PC_13_-27_1986_2020_summary.tif
/g/data/r78/datacube_stats/T

Unload all python modules, if you experience any issues.


In [22]:
%matplotlib widget
def show_widget(tile_label, subsample=5, continuous_update=True, mrvbf=False):
    tcw = get_tcw(tile_label).isel(band=3)[::subsample, ::subsample]
    tcg = get_tcg(tile_label).isel(band=1)[::subsample, ::subsample]
    
    initial_tcw_threshold = -600
    tcw_mask = tcw > initial_tcw_threshold
    initial_tcg_threshold = np.percentile(tcg, 95)
    tcg_mask = tcg < initial_tcg_threshold
    
    output = ipywidgets.Output()
    with output:
        fig = plt.figure(figsize=(6, 6))
        gs = fig.add_gridspec(ncols=2, nrows=2, width_ratios=[1, 1],
                              height_ratios=[1, 0.25], hspace=0.3, bottom=0.03, wspace=0.2,
                              top=0.95)
        ax = fig.add_subplot(gs[0, :])
        im = (tcw_mask & tcg_mask).plot.imshow(interpolation='bilinear', ax=ax)

        hist_ax_tcg = fig.add_subplot(gs[1, 0])
        hist_ax_tcg.set_title('TCG50')
        hist_tcg = hist_ax_tcg.hist(tcg.values.ravel(), bins=100, color='lightgray')
        hist_ax_tcw = fig.add_subplot(gs[1, 1])
        hist_ax_tcw.set_title('TCW95')
        hist_tcw = hist_ax_tcw.hist(tcw.values.ravel(), bins=100, color='lightgray')
        
        hist_line_tcg = hist_ax_tcg.axvline(initial_tcg_threshold, c='green')
        hist_line_tcw = hist_ax_tcw.axvline(initial_tcw_threshold, c='blue')

    def update_masks_and_draw():
        im.set_data(tcw_mask & tcg_mask)
        im.autoscale()
        fig.canvas.draw()
        
    def slider_update_green(change):
        with output:  # for errors
            threshold = change['new']
            if not isinstance(threshold, float):
                return

            nonlocal tcg_mask
            tcg_mask = tcg < threshold
            hist_line_tcg.set_xdata([threshold, threshold])

            update_masks_and_draw()

    def slider_update_wet(change):
        with output:  # for errors
            threshold = change['new']
            if not isinstance(threshold, float):
                return

            nonlocal tcw_mask
            tcw_mask = tcw > threshold
            hist_line_tcw.set_xdata([threshold, threshold])

            update_masks_and_draw()

    slider_props = dict(
        step=10,
        disabled=False,
        continuous_update=continuous_update,
        orientation='horizontal',
        readout=True,
        layout=ipywidgets.Layout(width='90%', margin='0 0 0 0'),
        style={'description_width': 'initial'},
        readout_format='.0f')
    slider_tcg = ipywidgets.FloatSlider(
        min=tcg.min(),
        max=tcg.max(),
        value=initial_tcg_threshold,
        description='Max TCG50:',
        **slider_props)
    slider_tcg.observe(slider_update_green, names='value')
    slider_tcw = ipywidgets.FloatSlider(
        min=tcw.min(),
        max=tcw.max(),
        value=initial_tcw_threshold,
        description='Max TCW95:',
        **slider_props)
    slider_tcw.observe(slider_update_wet, names='value')
    
    return ipywidgets.VBox([output, slider_tcg, slider_tcw])

In [23]:
show_widget('14,-21', subsample=5, continuous_update=True)